In [1]:
import os
import requests
import json

In [31]:
WB_TOKEN = os.getenv('WB_TOKEN')


def upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1):
  
    url = "https://content-api.wildberries.ru/content/v3/media/file"
    
    headers = {
        "Authorization": api_key,
        "X-Nm-Id": str(nm_id),
        "X-Photo-Number": str(photo_number)
    }
    
    with open(file_path, "rb") as f:
        files = {
            "uploadfile": (os.path.basename(file_path), f, "image/jpeg")
        }
        
        try:
            response = requests.post(url, headers=headers, files=files)
        except requests.RequestException as e:
            print(f"Ошибка при отправке запроса: {e}")
            return
        
        if response.status_code == 200:
            resp_json = response.json()
            if not resp_json.get("error", False):
                print(f"Успешно загружено изображение '{os.path.basename(file_path)}' на nmID={nm_id}.")
            else:
                print(f"Ошибка Wildberries при загрузке: {resp_json.get('errorText', 'Неизвестная ошибка')}")
        else:
            print(f"Ошибка ответа сервера. Код: {response.status_code}, файл: {os.path.basename(file_path)}")

def process_vendor_list(vendor_list, images_folder, api_key):
    """
    Проходит по списку vendor_list, проверяет, существует ли соответствующий файл в images_folder,
    и при наличии вызывает функцию загрузки на WB.
    
    :param vendor_list: Список словарей вида [{'nmID': ..., 'vendorCode': 'box_XXXX_YY'}, ...]
    :param images_folder: Путь к папке, где лежат файлы (например, 'images_900_1200').
    :param api_key: API-ключ для Wildberries.
    """
    for item in vendor_list:
        nm_id = item["nmID"]
        vendor_code = item["vendorCode"]
        
        # Ожидаем формат vendorCode = "box_9604677_30"
        # Удаляем "box_" в начале:
        if vendor_code.startswith("box_"):
            code_part = vendor_code[4:]  # "9604677_30"
            # Разделяем по символу "_"
            parts = code_part.split("_", maxsplit=1)
            if len(parts) == 2:
                product_id, quantity_str = parts
                # Формируем имя файла: "9604677-30.jpg"
                image_name = f"{product_id}-{quantity_str}.jpg"
                file_path = os.path.join(images_folder, image_name)
                
                # Проверяем, существует ли файл
                if os.path.isfile(file_path):
                    print(f"Файл '{image_name}' найден. Загружаем на nmID={nm_id}...")
                    upload_single_image_to_wb(file_path, nm_id, api_key, photo_number=1)
                else:
                    print(f"Файл '{image_name}' не найден в папке '{images_folder}'. Пропускаем.")
            else:
                print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")
        else:
            print(f"Неподходящий формат vendorCode: '{vendor_code}'. Пропускаем.")


In [32]:
all_cards_list = "https://content-api.wildberries.ru/content/v2/get/cards/list"
headers = {
    "Authorization": WB_TOKEN,
}
body =             {
          "settings": {                      
            "cursor": {
              "limit": 100
            },
            "filter": {
              "withPhoto": 0
            }
          }
        }


resp = requests.post(all_cards_list, headers=headers, json=body)

In [33]:
d = json.loads(resp.text)
d

{'cards': [], 'cursor': {'nmID': 0, 'total': 0}}

In [34]:
newNmIdVendorCodes = [{"nmID": row["nmID"], "vendorCode": row["vendorCode"]} for row in d["cards"]]

In [35]:
newNmIdVendorCodes

[]

In [36]:
process_vendor_list(newNmIdVendorCodes, "./images_900_1200", WB_TOKEN)